In [1]:
import torch
import torch.optim as optim
from torchinfo import summary

from KSI_models import KSI, ModifiedKSI, CAML
from KSI_utils import load_KSI_data, train_model, test_model

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
n_embedding = 100
n_hidden = 100 # 300 in paper, but too intensive for my machine
batch_size = 32
n_epochs = 25
save = True
profile = False
model_type = 'CAML'

In [3]:
dir = 'data/original/'
loaders, wikivec, word_to_ix = load_KSI_data(dir=dir, 
                                             batch_size=batch_size, 
                                             train=True, 
                                             val=True, 
                                             test=True, 
                                             device=DEVICE)
train_dataloader = loaders['train']
val_dataloader = loaders['val']
test_dataloader = loaders['test']

n_wiki, n_vocab = wikivec.shape
n_words = len(word_to_ix)

In [3]:
# note_lengths = []
# for data in train_dataloader:
#     n, _, _ = data
#     note_lengths.append(n.shape[1])
# avg_note_size = np.round(np.array(note_lengths).mean()).astype(int)

avg_note_size = 2455

In [5]:
base_model = CAML(n_words, n_wiki, n_embedding, n_hidden)
base_model = base_model.to(DEVICE)
base_summary = summary(base_model, [(batch_size, avg_note_size), 
                                    (batch_size, n_vocab)], 
                       dtypes=[torch.int, torch.float])

base_summary

Layer (type:depth-idx)                   Output Shape              Param #
CAML                                     --                        --
├─Embedding: 1-1                         [32, 2455, 100]           4,796,200
├─Dropout: 1-2                           [32, 2455, 100]           --
├─Conv1d: 1-3                            [32, 100, 2456]           100,100
├─Linear: 1-4                            --                        34,400
├─Linear: 1-5                            --                        34,744
Total params: 4,965,444
Trainable params: 4,965,444
Non-trainable params: 0
Total mult-adds (G): 8.02
Input size (MB): 1.87
Forward/backward pass size (MB): 125.72
Params size (MB): 19.86
Estimated Total Size (MB): 147.46

In [6]:
optimizer = optim.Adam(base_model.parameters())
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, 
                                          max_lr=0.01, 
                                          steps_per_epoch=len(train_dataloader), 
                                          epochs=n_epochs)
prof_base = train_model(base_model, 
                        train_dataloader=train_dataloader,
                        val_dataloader=val_dataloader,
                        optimizer=optimizer,
                        scheduler=scheduler,
                        n_epochs=n_epochs,
                        profile=profile, 
                        log_path=f'./log/{model_type}',
                        device=DEVICE)

Epoch: 001, Train Recall@10: 0.5400, Val Recall@10: 0.5413, Train Micro F1: 0.3866, Val Micro F1: 0.3877, Train Macro F1: 0.0192, Val Macro F1: 0.0232, Train Micro AUC: 0.9415, Val Micro AUC: 0.9289, Train Macro AUC: 0.6295, Val Macro AUC: 0.6378
Epoch: 002, Train Recall@10: 0.6906, Val Recall@10: 0.6807, Train Micro F1: 0.5660, Val Micro F1: 0.5541, Train Macro F1: 0.0677, Val Macro F1: 0.0797, Train Micro AUC: 0.9606, Val Micro AUC: 0.9513, Train Macro AUC: 0.7239, Val Macro AUC: 0.7244
Epoch: 003, Train Recall@10: 0.7604, Val Recall@10: 0.7453, Train Micro F1: 0.6210, Val Micro F1: 0.5964, Train Macro F1: 0.1184, Val Macro F1: 0.1313, Train Micro AUC: 0.9692, Val Micro AUC: 0.9601, Train Macro AUC: 0.7728, Val Macro AUC: 0.7495
Epoch: 004, Train Recall@10: 0.7750, Val Recall@10: 0.7557, Train Micro F1: 0.6315, Val Micro F1: 0.6065, Train Macro F1: 0.1432, Val Macro F1: 0.1575, Train Micro AUC: 0.9715, Val Micro AUC: 0.9624, Train Macro AUC: 0.7658, Val Macro AUC: 0.7508
Epoch: 005, 

In [7]:
if save:
    torch.save(base_model, f'{dir}{model_type}_model.pt')
if profile:
    print(prof_base.key_averages(group_by_stack_n=5).table(sort_by='self_cuda_time_total'))

In [8]:
tt_recall_at_k, tt_micro_f1, tt_macro_f1, tt_micro_auc, tt_macro_auc, label_aucs_base = test_model(base_model, 
                                                                                                   test_dataloader, 
                                                                                                   wikivec,
                                                                                                   device=DEVICE)
print(f'Test Recall@10: {tt_recall_at_k:.4f}, Test Micro F1: {tt_micro_f1:.4f}, Test Macro F1: {tt_macro_f1:.4f}' +
      f', Test Micro AUC: {tt_micro_auc:.4f}, Test Macro AUC: {tt_macro_auc:.4f}')
del base_model
if DEVICE == 'cuda':
    torch.cuda.empty_cache()

Test Recall@10: 0.7543, Test Micro F1: 0.6224, Test Macro F1: 0.1653, Test Micro AUC: 0.9557, Test Macro AUC: 0.6728


In [9]:
ksi = KSI(n_embedding, n_vocab)
ksi.to(DEVICE)
model = CAML(n_words, n_wiki, n_embedding, n_hidden, ksi=ksi)
model = model.to(DEVICE)
ksi_summary = summary(model, [(batch_size, avg_note_size), 
                              (batch_size, n_vocab),
                              (n_wiki, n_vocab)], 
                      dtypes=[torch.int, torch.float, torch.float])

ksi_summary

Layer (type:depth-idx)                   Output Shape              Param #
CAML                                     --                        --
├─KSI: 1-1                               --                        --
│    └─Linear: 2-1                       --                        (recursive)
│    └─Linear: 2-2                       --                        (recursive)
│    └─Linear: 2-3                       --                        (recursive)
├─Embedding: 1-2                         [32, 2455, 100]           4,796,200
├─Dropout: 1-3                           [32, 2455, 100]           --
├─Conv1d: 1-4                            [32, 100, 2456]           100,100
├─KSI: 1-1                               --                        --
│    └─Linear: 2-4                       [32, 344, 100]            1,217,400
│    └─Linear: 2-5                       [32, 344, 100]            10,100
│    └─Linear: 2-6                       [32, 344, 1]              101
Total params: 6,123,901
Trainable 

In [10]:
optimizer = optim.Adam(model.parameters())
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, 
                                          max_lr=0.01, 
                                          steps_per_epoch=len(train_dataloader), 
                                          epochs=n_epochs)
prof_ksi = train_model(model, 
                       train_dataloader=train_dataloader,
                       val_dataloader=val_dataloader,
                       wikivec=wikivec,
                       optimizer=optimizer,
                       scheduler=scheduler,
                       n_epochs=n_epochs, 
                       profile=profile, 
                       log_path=f'./log/{model_type}_KSI',
                       device=DEVICE)

Epoch: 001, Train Recall@10: 0.6643, Val Recall@10: 0.6612, Train Micro F1: 0.4511, Val Micro F1: 0.4464, Train Macro F1: 0.0500, Val Macro F1: 0.0604, Train Micro AUC: 0.9666, Val Micro AUC: 0.9589, Train Macro AUC: 0.8158, Val Macro AUC: 0.8108
Epoch: 002, Train Recall@10: 0.7594, Val Recall@10: 0.7463, Train Micro F1: 0.5769, Val Micro F1: 0.5610, Train Macro F1: 0.1080, Val Macro F1: 0.1205, Train Micro AUC: 0.9767, Val Micro AUC: 0.9689, Train Macro AUC: 0.8849, Val Macro AUC: 0.8532
Epoch: 003, Train Recall@10: 0.7928, Val Recall@10: 0.7743, Train Micro F1: 0.6312, Val Micro F1: 0.6038, Train Macro F1: 0.1766, Val Macro F1: 0.1829, Train Micro AUC: 0.9805, Val Micro AUC: 0.9717, Train Macro AUC: 0.9044, Val Macro AUC: 0.8600
Epoch: 004, Train Recall@10: 0.8068, Val Recall@10: 0.7816, Train Micro F1: 0.6537, Val Micro F1: 0.6186, Train Macro F1: 0.2270, Val Macro F1: 0.2099, Train Micro AUC: 0.9817, Val Micro AUC: 0.9724, Train Macro AUC: 0.8984, Val Macro AUC: 0.8525
Epoch: 005, 

In [11]:
if save:
    torch.save(model, f'{dir}{model_type}_KSI_model.pt')
if profile:
    print(prof_ksi.key_averages(group_by_stack_n=5).table(sort_by='self_cuda_time_total'))

In [12]:
tt_recall_at_k, tt_micro_f1, tt_macro_f1, tt_micro_auc, tt_macro_auc, label_aucs_ksi = test_model(model, 
                                                                                                  test_dataloader, 
                                                                                                  wikivec,
                                                                                                  device=DEVICE)
print(f'Test Recall@10: {tt_recall_at_k:.4f}, Test Micro F1: {tt_micro_f1:.4f}, Test Macro F1: {tt_macro_f1:.4f}' +
      f', Test Micro AUC: {tt_micro_auc:.4f}, Test Macro AUC: {tt_macro_auc:.4f}')
del model
if DEVICE == 'cuda':
    torch.cuda.empty_cache()

Test Recall@10: 0.7666, Test Micro F1: 0.6143, Test Macro F1: 0.2488, Test Micro AUC: 0.9687, Test Macro AUC: 0.8042


In [13]:
# run modified KSI using frequency vectors rather than binary vectors
dir = 'data/original_freqs/'
loaders, wikivec, word_to_ix = load_KSI_data(dir=dir, 
                                             batch_size=batch_size, 
                                             train=True, 
                                             val=True, 
                                             test=True, 
                                             device=DEVICE)
train_dataloader = loaders['train']
val_dataloader = loaders['val']
test_dataloader = loaders['test']

n_wiki, n_vocab = wikivec.shape
n_words = len(word_to_ix)

In [14]:
mod_ksi = ModifiedKSI(n_embedding, n_vocab)
mod_ksi.to(DEVICE)
mod_model = CAML(n_words, n_wiki, n_embedding, n_hidden, ksi=mod_ksi)
mod_model = mod_model.to(DEVICE)
mod_summary = summary(mod_model, [(batch_size, avg_note_size), 
                                  (batch_size, n_vocab),
                                  (n_wiki, n_vocab)], 
                      dtypes=[torch.int, torch.float, torch.float])

mod_summary

Layer (type:depth-idx)                   Output Shape              Param #
CAML                                     --                        --
├─ModifiedKSI: 1-1                       --                        --
│    └─Linear: 2-1                       --                        (recursive)
│    └─Linear: 2-2                       --                        (recursive)
│    └─Linear: 2-3                       --                        (recursive)
│    └─Linear: 2-4                       --                        (recursive)
├─Embedding: 1-2                         [32, 2455, 100]           4,796,200
├─Dropout: 1-3                           [32, 2455, 100]           --
├─Conv1d: 1-4                            [32, 100, 2456]           100,100
├─ModifiedKSI: 1-1                       --                        --
│    └─Linear: 2-5                       [32, 344, 12173, 1]       2
│    └─Linear: 2-6                       [32, 344, 100]            1,217,400
│    └─Linear: 2-7             

In [15]:
optimizer = optim.Adam(mod_model.parameters())
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, 
                                          max_lr=0.01, 
                                          steps_per_epoch=len(train_dataloader), 
                                          epochs=n_epochs)
prof_mod_ksi = train_model(mod_model, 
                           train_dataloader=train_dataloader,
                           val_dataloader=val_dataloader,
                           wikivec=wikivec,
                           optimizer=optimizer,
                           scheduler=scheduler,
                           n_epochs=n_epochs, 
                           profile=profile, 
                           log_path=f'./log/{model_type}_ModifiedKSI',
                           device=DEVICE)

Epoch: 001, Train Recall@10: 0.6667, Val Recall@10: 0.6642, Train Micro F1: 0.4905, Val Micro F1: 0.4871, Train Macro F1: 0.0525, Val Macro F1: 0.0643, Train Micro AUC: 0.9666, Val Micro AUC: 0.9597, Train Macro AUC: 0.8078, Val Macro AUC: 0.8363
Epoch: 002, Train Recall@10: 0.7579, Val Recall@10: 0.7516, Train Micro F1: 0.5694, Val Micro F1: 0.5602, Train Macro F1: 0.1086, Val Macro F1: 0.1298, Train Micro AUC: 0.9769, Val Micro AUC: 0.9711, Train Macro AUC: 0.8840, Val Macro AUC: 0.8804
Epoch: 003, Train Recall@10: 0.7902, Val Recall@10: 0.7825, Train Micro F1: 0.6188, Val Micro F1: 0.6006, Train Macro F1: 0.1573, Val Macro F1: 0.1719, Train Micro AUC: 0.9809, Val Micro AUC: 0.9749, Train Macro AUC: 0.9046, Val Macro AUC: 0.8938
Epoch: 004, Train Recall@10: 0.8014, Val Recall@10: 0.7870, Train Micro F1: 0.6327, Val Micro F1: 0.6094, Train Macro F1: 0.1913, Val Macro F1: 0.2013, Train Micro AUC: 0.9815, Val Micro AUC: 0.9749, Train Macro AUC: 0.8946, Val Macro AUC: 0.8742
Epoch: 005, 

In [16]:
if save:
    torch.save(mod_model, f'{dir}{model_type}_ModifiedKSI_model.pt')
if profile:
    print(prof_mod_ksi.key_averages(group_by_stack_n=5).table(sort_by='self_cuda_time_total'))

In [17]:
tt_recall_at_k, tt_micro_f1, tt_macro_f1, tt_micro_auc, tt_macro_auc, label_aucs_mod = test_model(mod_model, 
                                                                                                  test_dataloader, 
                                                                                                  wikivec,
                                                                                                  device=DEVICE)
print(f'Test Recall@10: {tt_recall_at_k:.4f}, Test Micro F1: {tt_micro_f1:.4f}, Test Macro F1: {tt_macro_f1:.4f}' +
      f', Test Micro AUC: {tt_micro_auc:.4f}, Test Macro AUC: {tt_macro_auc:.4f}')
del mod_model
if DEVICE == 'cuda':
    torch.cuda.empty_cache()

Test Recall@10: 0.7619, Test Micro F1: 0.6139, Test Macro F1: 0.2686, Test Micro AUC: 0.9677, Test Macro AUC: 0.8122


In [4]:
# run modified KSI using tfidf vectors rather than binary vectors
dir = 'data/original_tfidf/'
loaders, wikivec, word_to_ix = load_KSI_data(dir=dir, 
                                             batch_size=batch_size, 
                                             train=True, 
                                             val=True, 
                                             test=True, 
                                             device=DEVICE)
train_dataloader = loaders['train']
val_dataloader = loaders['val']
test_dataloader = loaders['test']

n_wiki, n_vocab = wikivec.shape
n_words = len(word_to_ix)

In [5]:
mod_ksi2 = ModifiedKSI(n_embedding, n_vocab)
mod_ksi2.to(DEVICE)
tfidf_model = CAML(n_words, n_wiki, n_embedding, n_hidden, ksi=mod_ksi2)
tfidf_model = tfidf_model.to(DEVICE)
tfidf_summary = summary(tfidf_model, [(batch_size, avg_note_size), 
                                      (batch_size, n_vocab),
                                      (n_wiki, n_vocab)], 
                        dtypes=[torch.int, torch.float, torch.float])

tfidf_summary

Layer (type:depth-idx)                   Output Shape              Param #
CAML                                     --                        --
├─ModifiedKSI: 1-1                       --                        --
│    └─Linear: 2-1                       --                        (recursive)
│    └─Linear: 2-2                       --                        (recursive)
│    └─Linear: 2-3                       --                        (recursive)
│    └─Linear: 2-4                       --                        (recursive)
├─Embedding: 1-2                         [32, 2455, 100]           4,796,200
├─Dropout: 1-3                           [32, 2455, 100]           --
├─Conv1d: 1-4                            [32, 100, 2456]           100,100
├─ModifiedKSI: 1-1                       --                        --
│    └─Linear: 2-5                       [32, 344, 12173, 1]       2
│    └─Linear: 2-6                       [32, 344, 100]            1,217,400
│    └─Linear: 2-7             

In [6]:
optimizer = optim.Adam(tfidf_model.parameters())
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, 
                                          max_lr=0.01, 
                                          steps_per_epoch=len(train_dataloader), 
                                          epochs=n_epochs)
prof_tfidf_ksi = train_model(tfidf_model, 
                             train_dataloader=train_dataloader,
                             val_dataloader=val_dataloader,
                             wikivec=wikivec,
                             optimizer=optimizer,
                             scheduler=scheduler,
                             n_epochs=n_epochs, 
                             profile=profile, 
                             log_path=f'./log/{model_type}_ModifiedKSI_tfidf',
                             device=DEVICE)

Epoch: 001, Train Recall@10: 0.6806, Val Recall@10: 0.6762, Train Micro F1: 0.4953, Val Micro F1: 0.4908, Train Macro F1: 0.0642, Val Macro F1: 0.0783, Train Micro AUC: 0.9681, Val Micro AUC: 0.9612, Train Macro AUC: 0.8252, Val Macro AUC: 0.8375
Epoch: 002, Train Recall@10: 0.7656, Val Recall@10: 0.7585, Train Micro F1: 0.5847, Val Micro F1: 0.5752, Train Macro F1: 0.1237, Val Macro F1: 0.1431, Train Micro AUC: 0.9779, Val Micro AUC: 0.9721, Train Macro AUC: 0.8872, Val Macro AUC: 0.8771
Epoch: 003, Train Recall@10: 0.7965, Val Recall@10: 0.7853, Train Micro F1: 0.6309, Val Micro F1: 0.6117, Train Macro F1: 0.1800, Val Macro F1: 0.1972, Train Micro AUC: 0.9815, Val Micro AUC: 0.9753, Train Macro AUC: 0.9105, Val Macro AUC: 0.8951
Epoch: 004, Train Recall@10: 0.8115, Val Recall@10: 0.7941, Train Micro F1: 0.6456, Val Micro F1: 0.6180, Train Macro F1: 0.2286, Val Macro F1: 0.2334, Train Micro AUC: 0.9833, Val Micro AUC: 0.9762, Train Macro AUC: 0.9100, Val Macro AUC: 0.8736
Epoch: 005, 

In [7]:
if save:
    torch.save(tfidf_model, f'{dir}{model_type}_ModifiedKSI_tfidf_model.pt')
if profile:
    print(prof_tfidf_ksi.key_averages(group_by_stack_n=5).table(sort_by='self_cuda_time_total'))

In [8]:
tt_recall_at_k, tt_micro_f1, tt_macro_f1, tt_micro_auc, tt_macro_auc, label_aucs_mod = test_model(tfidf_model, 
                                                                                                  test_dataloader, 
                                                                                                  wikivec,
                                                                                                  device=DEVICE)
print(f'Test Recall@10: {tt_recall_at_k:.4f}, Test Micro F1: {tt_micro_f1:.4f}, Test Macro F1: {tt_macro_f1:.4f}' +
      f', Test Micro AUC: {tt_micro_auc:.4f}, Test Macro AUC: {tt_macro_auc:.4f}')
del tfidf_model
if DEVICE == 'cuda':
    torch.cuda.empty_cache()

Test Recall@10: 0.7518, Test Micro F1: 0.6033, Test Macro F1: 0.2716, Test Micro AUC: 0.9652, Test Macro AUC: 0.8080
